In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer  # Required to enable IterativeImputer
from sklearn.impute import IterativeImputer
import joblib  # Import joblib for loading the model and preprocessing objects
test_file_path=
# Step 1: Load the new test data (numerical features only)
test_data = pd.read_csv(test_file_path)

# Step 2: Extract the 'ID' column and store it separately
ids = test_data['ID']

# Step 3: Drop unnecessary columns, including 'ID', same as done during training
columns_to_drop = ['ID', 'Column9', 'Column5', 'Column10', 'Column11', 'Column13', 
                   'Column14', 'Column15', 'Column16', 'Column20', 'Column21']
test_data = test_data.drop(columns=columns_to_drop)

# Step 4: Imputation of missing values using IterativeImputer
imputer = IterativeImputer(max_iter=10, random_state=0)
# Impute only numerical columns
numerical_columns = test_data.select_dtypes(include=['float64', 'int64']).columns
test_data_imputed = pd.DataFrame(imputer.fit_transform(test_data[numerical_columns]), columns=numerical_columns)

# Combine back non-numeric columns if any (though your dataset is numerical, in case future use has other columns)
test_data[test_data.select_dtypes(exclude=['float64', 'int64']).columns] = test_data.select_dtypes(exclude=['float64', 'int64'])

# Replace imputed data with the original test data
test_data.update(test_data_imputed)

# Step 5: Load the scaler and trained model
scaler = joblib.load('scaler.pkl')
model = joblib.load('catboost_model.pkl')  # Replace with your actual path

# Step 6: Apply scaling to the test data
scaled_test_data = scaler.transform(test_data)

# Step 7: Predict using the trained model
y_pred_new = model.predict(scaled_test_data)

# Step 8: Combine the 'ID' column with the predictions
predictions_df = pd.DataFrame({'ID': ids, 'target': y_pred_new})

# Step 9: Save the predictions to a CSV file
predictions_df.to_csv('predictions.csv', index=False)

print("Predictions with ID have been saved to predictions.csv")


Predictions with ID have been saved to predictions.csv
